## Timetabling Program for St. Margaret's School (ILP Only)

In [7]:
#import sys
#!{sys.executable} -m pip install ortools 

# import modules
import time
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

In [8]:
# Import Student Data
StudentInput = pd.ExcelFile("SMS Student Data.xlsx") 
StudentMatrix = pd.read_excel(StudentInput, 'Data') 
StudentInfo = StudentMatrix.values.tolist()
StudentMatrix[1:5]

,NAME,Grade,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10
1,S1,11,0,0,29,31,34,45,61,66,74,68
2,S2,11,16,0,29,34,47,55,61,71,74,0
3,S3,11,0,0,29,34,47,61,0,74,0,0
4,S4,11,0,29,34,46,52,63,68,75,0,45


In [9]:
# Import Course Data
CourseInput = pd.ExcelFile("SMS Course Data.xlsx") 
CourseMatrix = pd.read_excel(CourseInput, 'Data') 
CourseInfo = CourseMatrix.values.tolist()
CourseMatrix[1:5]

,Course Name,CourseID,Section,ABC,Teacher,FixedBlock,Core
1,Arts Education / ADST [Art Studio 11],3,1,NO,Louise,0,NO
2,Arts Education / ADST [Computer Programming 11],7,1,YES,William,0,NO
3,Arts Education / ADST [Culinary Arts 11],10,1,YES,Dana,0,NO
4,Arts Education / ADST [Culinary Arts 11],10,2,YES,Dana,0,NO


In [10]:
type(CourseMatrix)

pandas.core.frame.DataFrame

In [11]:
# Partition the offered courses into ShortCourses (ABC) and LongCourses (DEFGHI)
# We only consider the CourseID, not repeated sections of the course
ShortCourses=[]
LongCourses=[]
for m in range(len(CourseInfo)):
    CourseID = CourseInfo[m][1]
    if CourseInfo[m][2]==1:
        if CourseInfo[m][3]=="YES":
            ShortCourses.append(CourseID)
        else:
            LongCourses.append(CourseID)
            
# Partition the offered courses into OneSection and TwoSection
AllCourses=[CourseInfo[m][1] for m in range(len(CourseInfo))]
OneSection=[]
TwoSections=[]
for m in range(len(CourseInfo)):
    CourseID = CourseInfo[m][1]
    if CourseInfo[m][2]==2:
        TwoSections.append(CourseID)   
for CourseID in AllCourses:
    if not CourseID in TwoSections:
        OneSection.append(CourseID)
        
# Identify all Core Courses
CoreCourses=[]
for m in range(len(CourseInfo)):
    CourseID = CourseInfo[m][1]
    if CourseInfo[m][6]=="YES" and CourseInfo[m][2]==1:
        CoreCourses.append(CourseID)  

In [12]:
# Determine the preference coefficients based on the student data
# We have 10 points for a Core Course, 3 points for a Grade 12 elective
# and 1 point for a Grade 11 elective.

n = len(StudentInfo)
m = 100
P = np.zeros((n,m), dtype=int)
for i in range(n):
    for j in range(m):
        P[i,j]=-1

MaxScore=0
for i in range(n):
    for j in range(2,12):
        StudentGrade = StudentInfo[i][1]
        CoursePick = StudentInfo[i][j]
        if CoursePick > 0:
            if CoursePick in CoreCourses: P[i,CoursePick]=10
            else:
                if StudentGrade==12: P[i,CoursePick]=3
                if StudentGrade==11: P[i,CoursePick]=1

In [13]:
# Optimize short blocks

solver = pywraplp.Solver('St. Margarets School', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start_time = time.time()

AllStudents = range(n)

Sections = [1,2]
ShortBlocks = [1,2,3]

# define boolean variables
x = {}
for s in Sections:
    for j in ShortCourses:
        for k in ShortBlocks:
            x[s,j,k] = solver.IntVar(0,1, 'x[%d,%d,%d]' % (s,j,k))

y = {}
for i in AllStudents:
    for j in ShortCourses:
        for k in ShortBlocks:
            y[i,j,k] = solver.IntVar(0,1, 'y[%d,%d,%d]' % (i,j,k)) 


# CONSTRAINT 1: For two-section courses, ensure each section is offered once
for j in set(ShortCourses).intersection(TwoSections):
    solver.Add(sum(x[1,j,k] for k in ShortBlocks) == 1)
    solver.Add(sum(x[2,j,k] for k in ShortBlocks) == 1)
    
# CONSTRAINT 2: For single-section courses, ensure x[2,j,k] is set to 0
for j in set(ShortCourses).intersection(OneSection):
    solver.Add(sum(x[1,j,k] for k in ShortBlocks) == 1)
    solver.Add(sum(x[2,j,k] for k in ShortBlocks) == 0)

# CONSTRAINT 3: Two sections of the same course can't be offered in the same block
for j in ShortCourses:
    for k in ShortBlocks:
        solver.Add(x[1,j,k] + x[2,j,k] <= 1)

# CONSTRAINT 4: At most seven classes per block
for k in ShortBlocks:
    solver.Add(sum(x[1,j,k] + x[2,j,k] for j in ShortCourses) <= 7)

solver.Add(sum(x[1,j,1] + x[2,j,1] for j in ShortCourses) <= 6)        

# CONSTRAINT 5: No teacher can teach two classes in the same block
TeacherList = set([CourseInfo[z][4] for z in range(len(CourseInfo))])
for Teacher in TeacherList:
    ShortCourseList=[]
    for z in range(len(CourseInfo)):
        if CourseInfo[z][4]==Teacher and CourseInfo[z][1] in ShortCourses:
            ShortCourseList.append([CourseInfo[z][2],CourseInfo[z][1]])
    if len(ShortCourseList)>1:
        for k in ShortBlocks:
            solver.Add(sum(x[ShortCourseList[j][0],ShortCourseList[j][1],k] 
                           for j in range(len(ShortCourseList))) <= 1)
        
# CONSTRAINT 6: Ensure certain pre-defined courses are in fixed blocks
for z in range(len(CourseInfo)):
    if CourseInfo[z][5] in ShortBlocks:
        s = CourseInfo[z][2]
        j = CourseInfo[z][1]
        k = CourseInfo[z][5]
        solver.Add(x[s,j,k] == 1)
        
# CONSTRAINT 7: Each student takes at most one course per block
for k in ShortBlocks: 
    for i in AllStudents:
        solver.Add(sum(y[i,j,k] for j in ShortCourses) <= 1)

# CONSTRAINT 8: NO student can take the same course twice       
for j in ShortCourses:
    for i in AllStudents:
        solver.Add(sum(y[i,j,k] for k in ShortBlocks) <= 1)  

# CONSTRAINT 9: No student can take a course in a block when that course isn't offered
for i in AllStudents:
    for j in ShortCourses:
        for k in ShortBlocks:
            solver.Add(y[i,j,k] <= x[1,j,k]+x[2,j,k])
            
# CONSTRAINT 10: Every two-section course must have at most 18 students
for j in set(ShortCourses).intersection(TwoSections):
    for k in ShortBlocks:
            solver.Add(sum(y[i,j,k] for i in AllStudents) <= 18)
        
# CONSTRAINT 11: Do not assign course j to a student i if P[i,j]<0
for i in AllStudents:
    for j in ShortCourses:
        if P[i,j]<0:
            for k in ShortBlocks:
                solver.Add(y[i,j,k]==0)
                

solver.Maximize(solver.Sum(P[i,j]*y[i,j,k] for i in AllStudents for j in ShortCourses for k in ShortBlocks))
sol = solver.Solve()
print("")
print('Optimization Complete with Total Happiness Score of', round(solver.Objective().Value()))

# compute runtime
solving_time = time.time() - start_time

print('The code ran in', round(solving_time,1), 'seconds')



Optimization Complete with Total Happiness Score of 892
The code ran in 5.1 seconds


In [14]:
# Print the courses scheduled in each Short block and the number of
# students registered in each course.

CourseName = [0 for j in range(100)]
for i in range(len(CourseInfo)):
    CourseID = CourseInfo[i][1]
    CourseName[CourseID] = CourseInfo[i][0]

TeacherName = [0 for j in range(100)]
for i in range(len(CourseInfo)):
    CourseID = CourseInfo[i][1]
    TeacherName[CourseID] = CourseInfo[i][4]
    
ShortSchedule=[ [0,0,0] for i in AllStudents]
for i in AllStudents:
    for j in ShortCourses:
        for k in ShortBlocks:
            if y[i,j,k].solution_value()==1: 
                ShortSchedule[i][k-1]=j
                if j not in StudentInfo[i]:
                    print("ERROR", i,j,k)
                    
for k in ShortBlocks:
    print()
    print("Block", k, "courses are")

    for j in ShortCourses:
        if x[1,j,k].solution_value()==1:
            count=0
            for i in AllStudents:
                if ShortSchedule[i][k-1]==j:
                    count+=1
            print("Course", j, "Section 1 with", count, "students", CourseName[j], TeacherName[j])
        if x[2,j,k].solution_value()==1:
            count=0
            for i in AllStudents:
                if ShortSchedule[i][k-1]==j:
                    count+=1
            print("Course", j, "Section 2 with", count, "students", CourseName[j], TeacherName[j])


Block 1 courses are
Course 10 Section 1 with 1 students Arts Education / ADST [Culinary Arts 11] Dana
Course 16 Section 1 with 2 students Arts Education / ADST [Entrepreneurship and Marketing 10] Colin
Course 28 Section 2 with 17 students Career [Life Connections and Capstone 12] Kathy
Course 45 Section 1 with 14 students Combo Course Short Charlson
Course 75 Section 1 with 6 students Social Studies [Comparative Cultures 12] Sonny
Course 79 Section 1 with 6 students Social Studies [Philosophy 12] Justin

Block 2 courses are
Course 10 Section 2 with 2 students Arts Education / ADST [Culinary Arts 11] Dana
Course 29 Section 1 with 15 students Career [Life Education 10 Part B] Donna
Course 30 Section 1 with 7 students ELL Bridge [Composition 12] Nancy
Course 31 Section 1 with 14 students ELL Bridge [Spoken Language 11] Monica
Course 56 Section 1 with 1 students Modern Languages [Japanese Introductory 11] Michie
Course 59 Section 1 with 3 students Modern Languages [Spanish 12] Blair
Cours

In [15]:
# Optimize long blocks

solver = pywraplp.Solver('St. Margarets School', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start_time = time.time()

AllStudents = range(n)

Sections = [1,2]
LongBlocks = [4,5,6,7,8,9]

# define boolean variables
x = {}
for s in Sections:
    for j in LongCourses:
        for k in LongBlocks:
            x[s,j,k] = solver.IntVar(0,1, 'x[%d,%d,%d]' % (s,j,k))

y = {}
for i in AllStudents:
    for j in LongCourses:
        for k in LongBlocks:
            y[i,j,k] = solver.IntVar(0,1, 'y[%d,%d,%d]' % (i,j,k)) 

            
# CONSTRAINT 1: For two-section courses, ensure each section is offered once
for j in set(LongCourses).intersection(TwoSections):
    solver.Add(sum(x[1,j,k] for k in LongBlocks) == 1)
    solver.Add(sum(x[2,j,k] for k in LongBlocks) == 1)
    
# CONSTRAINT 2: For single-section courses, ensure x[2,j,k] is set to 0
for j in set(LongCourses).intersection(OneSection):
    solver.Add(sum(x[1,j,k] for k in LongBlocks) == 1)
    solver.Add(sum(x[2,j,k] for k in LongBlocks) == 0)

# CONSTRAINT 3: Two sections of the same course can't be offered in the same block
for j in LongCourses:
    for k in LongBlocks:
        solver.Add(x[1,j,k] + x[2,j,k] <= 1)

# CONSTRAINT 4: At most five classes per block
for k in LongBlocks:
    solver.Add(sum(x[1,j,k] + x[2,j,k] for j in LongCourses) <= 5)  
    solver.Add(sum(x[1,j,k] + x[2,j,k] for j in LongCourses) >= 4)  

solver.Add(sum(x[1,j,8] + x[2,j,8] for j in LongCourses) <= 4)

# CONSTRAINT 5: No teacher can teach two classes in the same block
TeacherList = set([CourseInfo[z][4] for z in range(len(CourseInfo))])
for Teacher in TeacherList:
    LongCourseList=[]
    for z in range(len(CourseInfo)):
        if CourseInfo[z][4]==Teacher and CourseInfo[z][1] in LongCourses:
            LongCourseList.append([CourseInfo[z][2],CourseInfo[z][1]])
    if len(LongCourseList)>1:
        for k in LongBlocks:
            solver.Add(sum(x[LongCourseList[j][0],LongCourseList[j][1],k] 
                           for j in range(len(LongCourseList))) <= 1)
        
# CONSTRAINT 6: Ensure certain pre-defined courses are in fixed blocks
for z in range(len(CourseInfo)):
    if CourseInfo[z][5] in LongBlocks:
        s = CourseInfo[z][2]
        j = CourseInfo[z][1]
        k = CourseInfo[z][5]
        solver.Add(x[s,j,k] == 1)
        
# CONSTRAINT 7: Each student takes at most one course per block
for k in LongBlocks: 
    for i in AllStudents:
        solver.Add(sum(y[i,j,k] for j in LongCourses) <= 1)

# CONSTRAINT 8: NO student can take the same course twice       
for j in LongCourses:
    for i in AllStudents:
        solver.Add(sum(y[i,j,k] for k in LongBlocks) <= 1)  

# CONSTRAINT 9: No student can take a course in a block when that course isn't offered
for i in AllStudents:
    for j in LongCourses:
        for k in LongBlocks:
            solver.Add(y[i,j,k] <= x[1,j,k]+x[2,j,k])
            
# CONSTRAINT 10: Every two-section course must have at most 18 students
for j in set(LongCourses).intersection(TwoSections):
    for k in LongBlocks:
            solver.Add(sum(y[i,j,k] for i in AllStudents) <= 18)
        
        
# CONSTRAINT 11: Do not assign course j to a student i if P[i,j]<0
for i in AllStudents:
    for j in LongCourses:
        if P[i,j]<0:
            for k in LongBlocks:
                solver.Add(y[i,j,k]==0)
                
            
# Extra constraints (remove once I've got my optimal schedule)
# And then fix up the Excel sheet to add my desired symmetry with Nancy, Sue etc. having FIXED blocks
# this reduces the running time significantly!

solver.Add(x[1,74,7]+x[1,74,8]+x[1,74,9]==1)
solver.Add(x[1,55,4]+x[1,55,5]+x[1,55,6]==1)
solver.Add(x[1,70,7]+x[1,70,8]+x[1,70,9]==0)
solver.Add(x[1,68,8]==1)
solver.Add(x[1,61,4]+x[1,61,5]+x[1,61,6]==1)
solver.Add(x[1,63,4]+x[1,63,5]+x[1,63,6]==1)



solver.Maximize(solver.Sum(P[i,j]*y[i,j,k] for i in AllStudents for j in LongCourses for k in LongBlocks))
sol = solver.Solve()
print("")
print('Optimization Complete with Total Happiness Score of', round(solver.Objective().Value()))

# compute runtime
solving_time = time.time() - start_time
print('The code ran in', round(solving_time,1), 'seconds')



Optimization Complete with Total Happiness Score of 1285
The code ran in 371.1 seconds


In [18]:
# Print the courses scheduled in each Long block and the number of
# students registered in each course.

LongSchedule=[ [0,0,0,0,0,0] for i in AllStudents]
for i in AllStudents:
    for j in LongCourses:
        for k in LongBlocks:
            if y[i,j,k].solution_value()==1: 
                LongSchedule[i][k-4]=j
                if j not in StudentInfo[i]:
                    print("ERROR", i,j,k)
                    
for k in LongBlocks:
    print()
    print("Block", k, "courses are")

    for j in LongCourses:
        if x[1,j,k].solution_value()==1:
            count=0
            for i in AllStudents:
                if LongSchedule[i][k-4]==j:
                    count+=1
            print("Course", j, "Section 1 with", count, "students", CourseName[j], TeacherName[j])
        if x[2,j,k].solution_value()==1:
            count=0
            for i in AllStudents:
                if LongSchedule[i][k-4]==j:
                    count+=1
            print("Course", j, "Section 2 with", count, "students", CourseName[j], TeacherName[j])
            

# NOTE: Course 34 Section 1 is taught by CAROL not NANCY so make the fix manually below


Block 4 courses are
Course 36 Section 2 with 7 students English Language Arts [English Studies 12] Nancy
Course 42 Section 1 with 12 students Mathematics [AP Calculus 12 AB] Charlson
Course 61 Section 1 with 17 students Physical Education [Active Living 11] Jeanine
Course 66 Section 1 with 13 students Science [Chemistry 11] Sue

Block 5 courses are
Course 3 Section 1 with 8 students Arts Education / ADST [Art Studio 11] Louise
Course 47 Section 2 with 12 students Mathematics [Pre-Calculus 12] Charlson
Course 55 Section 1 with 13 students Modern Languages [Japanese 12] Michie
Course 63 Section 1 with 11 students Physical Education [Fitness and Conditioning 11] Jeanine
Course 67 Section 1 with 4 students Science [Chemistry 12] Sue

Block 6 courses are
Course 2 Section 1 with 4 students Arts Education / ADST [AP Studio Art] Louise
Course 36 Section 1 with 18 students English Language Arts [English Studies 12] Nancy
Course 46 Section 1 with 7 students Mathematics [Pre-Calculus 11] William

In [19]:
# Print stats on the quality of our final timetable

CoreMax=0
Gr12Max=0
Gr11Max=0
CoreScore=0
Gr12Score=0
Gr11Score=0

for i in AllStudents:
    for j in range(m):
        if P[i,j]==10: 
            CoreMax+=1
            if j in ShortSchedule[i]: CoreScore+=1
            if j in LongSchedule[i]: CoreScore+=1
        if P[i,j]==3: 
            Gr12Max+=1
            if j in ShortSchedule[i]: Gr12Score+=1
            if j in LongSchedule[i]: Gr12Score+=1
        if P[i,j]==1: 
            Gr11Max+=1
            if j in ShortSchedule[i]: Gr11Score+=1
            if j in LongSchedule[i]: Gr11Score+=1
                
print("TOTAL: Students got into", CoreScore, "out of", CoreMax, "core courses")
print("Grade 12s got into", Gr12Score, "out of", Gr12Max, "elective courses")
print("Grade 11s got into", Gr11Score, "out of", Gr11Max, "elective courses")

TOTAL: Students got into 167 out of 167 core courses
Grade 12s got into 122 out of 129 elective courses
Grade 11s got into 141 out of 151 elective courses


In [20]:
# Print list of all students who didn't get into ALL of their desired courses
# and how many courses that missed getting into

for i in AllStudents:
    choices=0
    for j in range(2,12):
        if StudentInfo[i][j]>0:
            choices+=1
    for j in ShortSchedule[i]:
        if j>0:
            choices-=1
    for j in LongSchedule[i]:
        if j>0:
            choices-=1
    if choices>0:
        missedlist = set(StudentInfo[i]) - set(ShortSchedule[i]) - set(LongSchedule[i]) - {11,12} - {StudentInfo[i][0]}
        print("Student", i, "missed", choices, "-", ShortSchedule[i],LongSchedule[i], missedlist)

Student 1 missed 1 - [45, 31, 29] [61, 0, 0, 34, 68, 66] {74}
Student 4 missed 1 - [45, 29, 0] [0, 63, 46, 34, 68, 52] {75}
Student 9 missed 1 - [0, 31, 29] [61, 0, 71, 47, 34, 66] {76}
Student 18 missed 1 - [0, 0, 29] [0, 63, 71, 34, 46, 66] {70}
Student 19 missed 1 - [45, 29, 0] [61, 0, 71, 34, 68, 66] {52}
Student 20 missed 1 - [0, 31, 29] [66, 3, 46, 0, 34, 0] {53}
Student 24 missed 1 - [0, 31, 29] [61, 0, 71, 34, 46, 66] {59}
Student 26 missed 1 - [45, 31, 29] [66, 63, 46, 34, 68, 0] {3}
Student 28 missed 1 - [45, 31, 29] [66, 63, 0, 47, 34, 0] {68}
Student 30 missed 1 - [0, 78, 29] [66, 47, 71, 38, 0, 0] {59}
Student 35 missed 1 - [28, 0, 0] [42, 55, 36, 65, 67, 0] {72}
Student 38 missed 1 - [28, 30, 76] [36, 55, 71, 65, 0, 81] {47}
Student 44 missed 1 - [28, 0, 7] [42, 47, 36, 72, 67, 81] {76}
Student 47 missed 1 - [10, 30, 28] [0, 67, 36, 65, 46, 52] {81}
Student 53 missed 1 - [28, 0, 0] [42, 55, 36, 65, 67, 0] {72}
Student 54 missed 1 - [28, 31, 0] [0, 47, 36, 72, 67, 81] {78}